# USB Inrush Current limiting
We have about 20 uF of capacitance at our input.
In order to limit the inrush current.
The USB power supply is considered an ideal voltage source.
- max 10 uF for inrush current, we have about 20 uF
- The DC resistance of the ferrite is about 0.04 - 0.05 DCR
- Add a resistor just to be sure

**Conclusion**
I don't think the inrush current will be a big problem, however if it is we can always add a small resistor of about 1-2 ohms in series (0805 pad prepared here)

# CC1/2 Voltage Supply detection

Goal here is to build 2 shmitt-triggers that will drive a LED depending on which CC1 is active and whether the CC1 voltage is above 0.4 Volts.
So we need to the same shmitt triggers, each one with an input voltage coming from the CC1 pin which will indicate it's capabilities through the source resistance.

- The cheaper / more common comparators haven an open-drain output.
- They can thus only source current.

We however definitely need one comparator that has a push-pull output in order to drive the switch, if we don't want to add any extra components (like transistors) to the output
### the DIO2087x
- Comes in an SOT-23-5 package (quite common for single-diode)
	- However doesn't seem to be able to sink / source enough current
	- No simulation option available

Alternative:

### COS3201
Source and sinking current: 57 mA
- Ioffset: 1-5 mV
- Ibias: 1-10 pA
- Ioffset: 1-10 pA

![](images/non_inverting_hist.png)

### Purpose of R3
Compensates for bias current and setting the reference voltage.

- Opamps have a finite input resistance.
- Bias current flows to the input and creates a voltage drop across R1 and R2.
	- This introduces an error making the hysteresis less accurate
- The central solution to this is to make the non-inverting and inverting inputs have the same source impedance
- Ideally: equivalent resistnace at the non-inverting input should match the resistance at the inverting input

Thevenin resistance of reference voltage: 
- Rsource || Rsink (max: 5 kOhm || 51 kOhm = 4.55 kOhm), so the R1 has to be at least order of magnitude 10 of that to minimize interactions of schmitt trigger with source impedance of Vin

Vref: 0.7 V, Relatively low impedance voltage divider can be chosen.

In [61]:
Vref = 0.7 # V
Vcc = 5 # V
Ibias = 10e-12
If = 10 * Ibias

#### Chooose R3 
# (Current through R3 should be > input bias current)
R3_1 = Vref / If
R3_2 = (Vcc - Vref) / If
R3_min = min(R3_1, R3_2)
R3_ch = 2e6

print(f"R3 = {R3_min*1e-6:.2f} MOhm, chosen R3 = {R3_ch*1e-6:.2f} MOhm")

#### Choose hysteresis band
Vhb = 0.2 # V
R1_ch = R3_ch * (Vhb / Vcc)

#### Choose trip point for Vin
Vthr = 0.85 # V
R2_ch = 1 / ((Vthr / (Vref * R1_ch)) - 1 / R1_ch - 1 / R3_ch)

Vthr = Vref * R1_ch * (1/R1_ch + 1 / R2_ch + 1 / R3_ch)
Vthf = Vthr - (R1_ch * Vcc) / R3_ch
hyst = Vthr - Vthf
print(f"Vthr: {Vthr}, Vthf: {Vthf}, Hys: {hyst}")
print(f"R1: {R1_ch*1e-3:.2f} kOhm, R2: {R2_ch*1e-3:.2f} kOhm, R3: {R3_ch*1e-3:.2f} kOhm")

R3 = 7000.00 MOhm, chosen R3 = 2.00 MOhm
Vthr: 0.85, Vthf: 0.6499999999999999, Hys: 0.20000000000000007
R1: 80.00 kOhm, R2: 459.02 kOhm, R3: 2000.00 kOhm


### R3 Thoughts
It seems like R3 is interfering more with the biasing network than actually solving Ibias issues.
On top of that Ibias is not really that big of a problem, since it is barely a few pA.
So let's simply use 2 bias resistors, and make the resistive divider from the CC-pins as small as possible, so it's source impedance is negligible compared to 

- https://www.ti.com/lit/ds/snos532o/snos532o.pdf?ts=1749333508860&ref_url=https%253A%252F%252Fwww.google.com%252F

$$dV_{in} = V_{cc} * R1 / R2$$

In [114]:
Vref = 0.8 # V
Vcc = 5 # V
Ibias = 10e-12
dVin = 0.3

#### Choose hysteresis band
R1_R2_ratio = dVin / Vcc
R2 = 2e6 # 2 Mega-Ohms
R1 = R2 *  R1_R2_ratio
print(f"R1: {R1*1e-3:.2f}kOhm, R2: {R2*1e-3:.2f}kOhm")

Vin1 = Vref * (R1+R2) / R2
Vin2 =(Vref* (R1+R2) - Vcc * R1) / R2
print(f"Bounds: {Vin2:.2f} V .. {Vin1:.2f} V")

R1: 120.00kOhm, R2: 2000.00kOhm
Bounds: 0.55 V .. 0.85 V


Simulate using the TS7211
- Ibias: 1 pA normally (up to 300 pA)
- Ioffset: 1 pA normally (up to 300 pA)
- Vo max a few mV

### Resistor choice
Largest resistors:
- 10 MOhm: C7250
- 2 MOhm: C22976
- 1 MOhm: C22935

### Led choice
Choose an LED: KT-0603R, KT-0805Y (the other ones have a voltage drop that varies too much up to 3 V)
- Choose the defaults by JLCPCB
	- Default breakdown voltage of 5 V
	- Forward voltage drop of 2 V

### Simulate relevant source impedances for signals

#### Input signal
- Voltage divider of (at worst) 5.1 kOhm and 51 kOhm for Vin
- Vin should vary between
	- 0.45 V
	- 0.94 V

And should saturate when reaching 0.85 V, the opposite should happen at 0.45 V

#### Effect of additional source resistance
- Additional source resistance for the signal adds onto the input resistance for the schmitt trigger.
- This leads to the effective hysteresis voltage as seen by the source becoming larger (since dVin = R1/R2 * Vcc)

#### Reference signal
Should be 0.7 V for the CC-detection

So in the voltage divider R2/(R1+R2) = Vref / Vcc
So: Vcc / Vref - 1 = R1 / R2 = 5 / 0.7 - 1 = 6.143

Choose resistors 6.2 kOhm and 1 kOhm (670 uA) for both inputs


In [111]:
Vcc = 5
Vref = 0.8
R1_R2_ratio = Vcc / Vref - 1
print(f"Ratio: {R1_R2_ratio}")

import numpy as np

resistor_array = np.array([0.470, 0.51, 0.56, 0.68, 0.82, 1, 1.2, 1.5, 1.8, 2, 2.2, 2.4, 2.7, 3, 3.3, 3.6, 3.9, 4.7, 4.99, 5.1, 5.6, 6.2, 
						6.8, 7.5, 8.2, 9.1, 10, 12, 13, 15, 20, 22, 24, 27, 30, 33, 39, 47, 51, 56, 68, 75, 82, 100, 150])

######### OPAMP INPUT
v_target_ratio = R1_R2_ratio # R1/R2
r2_target_arr = np.round(resistor_array / v_target_ratio, 2)
r1_target_arr = np.round(resistor_array * v_target_ratio, 2)
print(f"r1/r2: {v_target_ratio}")
print(f"R1 values {r1_target_arr}")
print(f"R2 values {r2_target_arr}")

for r1, r2 in zip(r1_target_arr, r1_target_arr):
	if (r2 in resistor_array):
		print(f"Perfect r2 found: {r2}")
	if (r1 in resistor_array):
		print(f"Perfect r1 found: {r1}")
# Choose 6.2 kOhm R1, 1.2 kOhm R2

Ratio: 5.25
r1/r2: 5.25
R1 values [  2.47   2.68   2.94   3.57   4.3    5.25   6.3    7.88   9.45  10.5
  11.55  12.6   14.18  15.75  17.32  18.9   20.47  24.68  26.2   26.78
  29.4   32.55  35.7   39.38  43.05  47.78  52.5   63.    68.25  78.75
 105.   115.5  126.   141.75 157.5  173.25 204.75 246.75 267.75 294.
 357.   393.75 430.5  525.   787.5 ]
R2 values [ 0.09  0.1   0.11  0.13  0.16  0.19  0.23  0.29  0.34  0.38  0.42  0.46
  0.51  0.57  0.63  0.69  0.74  0.9   0.95  0.97  1.07  1.18  1.3   1.43
  1.56  1.73  1.9   2.29  2.48  2.86  3.81  4.19  4.57  5.14  5.71  6.29
  7.43  8.95  9.71 10.67 12.95 14.29 15.62 19.05 28.57]


In [113]:
R1_chosen = 100e3
R2_chosen = 2e6
Vin1 = Vref * (R1_chosen+R2_chosen) / R2_chosen
Vin2 = (Vref*(R1_chosen+R2_chosen) - Vcc * R1_chosen) / R2_chosen
print(f"Bounds: {Vin2:.2f} V .. {Vin1:.2f} V")

Bounds: 0.59 V .. 0.84 V


# Comparator for analog USB switching purposes

- Hysteresis must be 0.2 V


In [117]:
Vcc = 5 # V
Ibias = 10e-12
dVin = 0.2
Vth_high = 0.3

#### Choose hysteresis band
R1_R2_ratio = dVin / Vcc
print(f"R1_R2_ratio: {R1_R2_ratio}")
R2 = 1e6 # 2 Mega-Ohms
R1 = R2 *  R1_R2_ratio
print(f"R1: {R1*1e-3:.2f}kOhm, R2: {R2*1e-3:.2f}kOhm")

Vref = Vth_high / (1 + R1_R2_ratio)
print(f"Vref: {Vref}")
Vth_low = (Vref*(R1+R2) - Vcc * R1) / R2
print(f"Bounds: {Vth_low:.2f} V .. {Vth_high:.2f} V")

R1_R2_ratio: 0.04
R1: 40.00kOhm, R2: 1000.00kOhm
Vref: 0.28846153846153844
Bounds: 0.10 V .. 0.30 V


## Getting Vref: 0.288 V

In [116]:
Vcc = 5
Vref = 0.288
R1_R2_ratio = Vcc / Vref - 1
print(f"Ratio: {R1_R2_ratio}")

import numpy as np

resistor_array = np.array([0.470, 0.51, 0.56, 0.68, 0.82, 1, 1.2, 1.5, 1.8, 2, 2.2, 2.4, 2.7, 3, 3.3, 3.6, 3.9, 4.7, 4.99, 5.1, 5.6, 6.2, 
						6.8, 7.5, 8.2, 9.1, 10, 12, 13, 15, 20, 22, 24, 27, 30, 33, 39, 47, 51, 56, 68, 75, 82, 100, 150])

######### OPAMP INPUT
v_target_ratio = R1_R2_ratio # R1/R2
r2_target_arr = np.round(resistor_array / v_target_ratio, 2)
r1_target_arr = np.round(resistor_array * v_target_ratio, 2)
print(f"r1/r2: {v_target_ratio}")
print(f"R1 values {r1_target_arr}")
print(f"R2 values {r2_target_arr}")

for r1, r2 in zip(r1_target_arr, r1_target_arr):
	if (r2 in resistor_array):
		print(f"Perfect r2 found: {r2}")
	if (r1 in resistor_array):
		print(f"Perfect r1 found: {r1}")
# Choose R1 = 9.1 kOhm, R2 = 560 Ohm

Ratio: 16.36111111111111
r1/r2: 16.36111111111111
R1 values [   7.69    8.34    9.16   11.13   13.42   16.36   19.63   24.54   29.45
   32.72   35.99   39.27   44.18   49.08   53.99   58.9    63.81   76.9
   81.64   83.44   91.62  101.44  111.26  122.71  134.16  148.89  163.61
  196.33  212.69  245.42  327.22  359.94  392.67  441.75  490.83  539.92
  638.08  768.97  834.42  916.22 1112.56 1227.08 1341.61 1636.11 2454.17]
R2 values [0.03 0.03 0.03 0.04 0.05 0.06 0.07 0.09 0.11 0.12 0.13 0.15 0.17 0.18
 0.2  0.22 0.24 0.29 0.3  0.31 0.34 0.38 0.42 0.46 0.5  0.56 0.61 0.73
 0.79 0.92 1.22 1.34 1.47 1.65 1.83 2.02 2.38 2.87 3.12 3.42 4.16 4.58
 5.01 6.11 9.17]


## Schmit trigger load impedance
![](images/schmitt_trigger.webp)

So there's 2 options
1. Vout is HIGH (5 V)
2. Vout is low (ground)

## Lowest impedance
Lowest impedance = critical point, load impedance is low and cc voltage rissk dropping too low.

### Kirchoff
- Vout = 0 V
- Vin: input terminal voltage
- (Vin - Vout) / (R1+R2) = Iin (assume no current goes into the + terminal)
	- Impedance is thus R1 + R2 approximately.


## Negative impedance
Negative impedance: cc voltage risks becoming too high due to additional voltage drop from current.
- Vout = 5 V
- Vin: input terminal voltage (0.94 V, 1.69 V)
- Calculate current flowing to source, given Rs is 22 kOhm // 5.1 kOhm = 4.1 kOhm
	- (0.94 - 5) / (R1 + R2 + Rs) = 3.85 uA
- Calculate current flowing to source, given Rs is 10 kOhm // 5.1 kOhm = 3.38kOhm
	- (1.69 - 5) / (R1 + R2 + Rs) = 3.85 uA
	- Voltage drop over a 5.1 kOhm resistor becomes: 20 mV -> Which is quite a big voltage drop, (difference between 1.69 and 1.67) -> 1.1 %


## Better option
- Use a single buffer to buffer the usb_cc_vref voltage
- Apply the voltages USB_CC1 and USB_CC2 to the inverting terminals

This way you only need a SINGLE buffer (for your reference voltage) and the CC-voltages simply see the high impedance non-inverting input.

2 Options:
- Vout: 5 V (Vcc)
	- non-inverting input (assume Rs = 0): 
		- (Vref - 5) / (R2 + R1) = IR1
		- Vref - IR1 * R1 = V+ 
			- [IR1 < 0 => V+ is large]
		- When V+ > Vin: output is 5 V (Vcc)
 - Vout: 0 V (GND)
	- non-inverting input (assume Rs = 0):
		- Vref / (R2+R1) = IR1
		- Vref - IR1 * R1 = V+
			- [IR1 > 0 => V+ is small]
		- When V+ < Vin: output is 0 (GND)